In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack, csr_matrix
import numpy as np

class TextRelevanceModel:
    def __init__(self, keyword_categories, negative_keywords=None, model=None):
        self.keyword_categories = keyword_categories
        self.negative_keywords = negative_keywords if negative_keywords is not None else []
        self.vectorizer = TfidfVectorizer()
        self.model = model if model is not None else LogisticRegression(max_iter=1000, random_state=42)
    
    def keyword_count(self, text, keywords):
        return sum(1 for word in text.lower().split() if word in keywords)
    
    def prepare_data(self, df, text_column):
        df[text_column] = df[text_column].str.lower()

        keyword_counts = pd.DataFrame()
        
        for category, keywords in self.keyword_categories.items():
            keyword_counts[category + '_count'] = df[text_column].apply(lambda x: self.keyword_count(x, keywords))
        
        keyword_counts['negative_keyword_count'] = df[text_column].apply(lambda x: self.keyword_count(x, self.negative_keywords))
        
        X_text = self.vectorizer.fit_transform(df[text_column])
        X_keywords = keyword_counts.to_numpy()
        X = hstack([X_text, csr_matrix(X_keywords)])
        
        return X
    
    def train(self, X, y):
        keyword_present = np.any(X[:, -len(self.keyword_categories):].toarray(), axis=1)
        X_train = X[keyword_present]
        y_train = y[keyword_present]
        
        self.model.fit(X_train, y_train)
    
    def predict_proba(self, text):
        X_text_new = self.vectorizer.transform([text])
        keyword_counts_new = np.array([[self.keyword_count(text, keywords) for keywords in self.keyword_categories.values()]])
        negative_keyword_count_new = np.array([[self.keyword_count(text, self.negative_keywords)]])
        X_new = hstack([X_text_new, csr_matrix(keyword_counts_new), csr_matrix(negative_keyword_count_new)])
        
        if not np.any(keyword_counts_new):
            return 0.0
        
        return self.model.predict_proba(X_new)[0, 1]


Probability of relevance: 0.49353904502514173


In [19]:
# Example usage
if __name__ == "__main__":
    # Example DataFrame
    data = {
        'text': [
            'This is the first document.', 'This document is the second document.', 
            'And this is the third one.', 'Is this the first document?',
            'More text data.', 'Even more text.', 'Text data again.', 
            'And another one.', 'More examples.', 'Additional text.',
            'Sample text.', 'Another sample.', 'More samples.', 'Final example.'
        ],
        'relevance': [1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0]  # Relevance labels
    }
    df = pd.DataFrame(data)
    
    # Define categories of keywords
    keyword_categories = {
        'Category1': ['first', 'second', 'third'],
        'Category2': ['more', 'additional', 'sample']
    }
    
    # Define negative keywords
    negative_keywords = ['another', 'final']
    
    # Create an instance of the model
    model = TextRelevanceModel(keyword_categories, negative_keywords)
    
    # Prepare data
    X = model.prepare_data(df, text_column='text')
    y = df['relevance']
    
    # Train the model
    model.train(X, y)
    
    # Predict the probability of relevance for a new sample of text
    text_sample = "This is a new document with some more relevant keywords"
    relevance_probability = model.predict_proba(text_sample)
    print("Probability of relevance:", relevance_probability)


Probability of relevance: 0.49353904502514173


In [20]:
# Example DataFrame with 5 text samples
data = {
    'text': [
        'This is the first document.', 
        'This document is the second document.', 
        'And this is the third one.', 
        'Is this the first document?',
        'More text data.'
    ]
}
df_samples = pd.DataFrame(data)

# Predict the probability of relevance for each text sample
for i, text_sample in enumerate(df_samples['text']):
    relevance_probability = model.predict_proba(text_sample)
    print(f"Probability of relevance for sample {i+1}: {relevance_probability:.2f}")


Probability of relevance for sample 1: 0.36
Probability of relevance for sample 2: 0.36
Probability of relevance for sample 3: 0.34
Probability of relevance for sample 4: 0.36
Probability of relevance for sample 5: 0.63


In [21]:
# Example usage
if __name__ == "__main__":
    # Importing and dropping rows from Frame

    df_coded = pd.read_csv('../data/processed_and_coded_posts.csv')
    df = df_coded[['processed_text','highly_relevant']]
    
    #Importing keywords

    csv_file_path = '../keywords/medications.csv'

    # Read the CSV file
    df_med = pd.read_csv(csv_file_path)

    # Extract the first column as a list of keywords
    medications = df_med.iloc[:, 0].tolist()

    csv_file_path_2 = '../keywords/Treatment.csv'

    # Read the CSV file
    df_therapy = pd.read_csv(csv_file_path_2)

    # Extract the first column as a list of keywords
    therapy = df_therapy.iloc[:, 0].tolist()

    general_keywords = ['medicine','therapy','treatment','recovery','prescribed','diagnosed','med','meds','prescribe','therapist','session','psychiatrist','psychiatrists','dosage','medication', 'dbt', 'abilify', 'outpatient', 'therapist', 'harming','medicine','therapy','treatment','recovery','prescribed','diagnosed','therapists','prescribe','diagnose','medicines','drugs','drug','therapist','session']

    # Define categories of keywords
    
    keyword_categories = {
    'general_keywords': general_keywords,
    'medications': medications,
    'therapy': therapy
}
    
    # Define negative keywords
    negative_keywords = ['relationship', 'friend', 'together', 'fp', 'people', 'person', 'partner', 'dating']
    
    # Create an instance of the model
    model = TextRelevanceModel(keyword_categories, negative_keywords)
    
    #Prepare data
    X = model.prepare_data(df, text_column='processed_text')
    y = df['highly_relevant']

    # Train the model
    model.train(X, y)


    # Predict the probability of relevance for a new sample of text
    text_sample = "This is a new an excerpt on dbt and emdr"
    relevance_probability = model.predict_proba(text_sample)
    print("Probability of relevance:", relevance_probability)

   

/var/folders/9j/j4gyltgs6txd83bg66_mv4fn664jzt/T/ipykernel_32169/2104515649.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[text_column] = df[text_column].str.lower()


Probability of relevance: 0.09328291951431598


In [11]:
df.columns

Index(['processed_text', 'highly_relevant'], dtype='object')

In [32]:
# Predict the probability of relevance for each text sample
df_coded['relevance_probability']=0.0

for i, text_sample in enumerate(df['processed_text']):
    relevance_probability = model.predict_proba(text_sample)
    df_coded['relevance_probability'][i]=relevance_probability
    print(f"Probability of relevance for sample {i+1}: {relevance_probability:.2f}")

/var/folders/9j/j4gyltgs6txd83bg66_mv4fn664jzt/T/ipykernel_32169/3805266683.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_coded['relevance_probability'][i]=relevance_probability
/var/folders/9j/j4gyltgs6txd83bg66_mv4fn664jzt/T/ipyke

Probability of relevance for sample 1: 0.00
Probability of relevance for sample 2: 0.00
Probability of relevance for sample 3: 0.00
Probability of relevance for sample 4: 0.04
Probability of relevance for sample 5: 0.00
Probability of relevance for sample 6: 0.06
Probability of relevance for sample 7: 0.00
Probability of relevance for sample 8: 0.00
Probability of relevance for sample 9: 0.00
Probability of relevance for sample 10: 0.19
Probability of relevance for sample 11: 0.09
Probability of relevance for sample 12: 0.02
Probability of relevance for sample 13: 0.00
Probability of relevance for sample 14: 0.00
Probability of relevance for sample 15: 0.05
Probability of relevance for sample 16: 0.01
Probability of relevance for sample 17: 0.00
Probability of relevance for sample 18: 0.09
Probability of relevance for sample 19: 0.00
Probability of relevance for sample 20: 0.00
Probability of relevance for sample 21: 0.00
Probability of relevance for sample 22: 0.00
Probability of rele

/var/folders/9j/j4gyltgs6txd83bg66_mv4fn664jzt/T/ipykernel_32169/3805266683.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_coded['relevance_probability'][i]=relevance_probability
/var/folders/9j/j4gyltgs6txd83bg66_mv4fn664jzt/T/ipyke

Probability of relevance for sample 28: 0.00
Probability of relevance for sample 29: 0.01
Probability of relevance for sample 30: 0.00
Probability of relevance for sample 31: 0.04
Probability of relevance for sample 32: 0.01
Probability of relevance for sample 33: 0.29
Probability of relevance for sample 34: 0.02
Probability of relevance for sample 35: 0.11
Probability of relevance for sample 36: 0.00
Probability of relevance for sample 37: 0.01
Probability of relevance for sample 38: 0.00
Probability of relevance for sample 39: 0.00
Probability of relevance for sample 40: 0.00
Probability of relevance for sample 41: 0.00
Probability of relevance for sample 42: 0.00
Probability of relevance for sample 43: 0.00
Probability of relevance for sample 44: 0.00
Probability of relevance for sample 45: 0.00
Probability of relevance for sample 46: 0.00
Probability of relevance for sample 47: 0.05
Probability of relevance for sample 48: 0.08
Probability of relevance for sample 49: 0.00
Probabilit

/var/folders/9j/j4gyltgs6txd83bg66_mv4fn664jzt/T/ipykernel_32169/3805266683.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_coded['relevance_probability'][i]=relevance_probability
/var/folders/9j/j4gyltgs6txd83bg66_mv4fn664jzt/T/ipyke

...

/var/folders/9j/j4gyltgs6txd83bg66_mv4fn664jzt/T/ipykernel_32169/3805266683.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_coded['relevance_probability'][i]=relevance_probability
/var/folders/9j/j4gyltgs6txd83bg66_mv4fn664jzt/T/ipyke

Probability of relevance for sample 540: 0.05
Probability of relevance for sample 541: 0.00
Probability of relevance for sample 542: 0.00
Probability of relevance for sample 543: 0.00
Probability of relevance for sample 544: 0.00
Probability of relevance for sample 545: 0.00
Probability of relevance for sample 546: 0.00
Probability of relevance for sample 547: 0.00
Probability of relevance for sample 548: 0.44
Probability of relevance for sample 549: 0.00
Probability of relevance for sample 550: 0.00
Probability of relevance for sample 551: 0.00
Probability of relevance for sample 552: 0.00
Probability of relevance for sample 553: 0.02
Probability of relevance for sample 554: 0.00
Probability of relevance for sample 555: 0.00
Probability of relevance for sample 556: 0.00
Probability of relevance for sample 557: 0.00
Probability of relevance for sample 558: 0.00
Probability of relevance for sample 559: 0.01
Probability of relevance for sample 560: 0.00
Probability of relevance for sampl

/var/folders/9j/j4gyltgs6txd83bg66_mv4fn664jzt/T/ipykernel_32169/3805266683.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_coded['relevance_probability'][i]=relevance_probability
/var/folders/9j/j4gyltgs6txd83bg66_mv4fn664jzt/T/ipyke

Probability of relevance for sample 572: 0.00
Probability of relevance for sample 573: 0.05
Probability of relevance for sample 574: 0.98
Probability of relevance for sample 575: 0.00
Probability of relevance for sample 576: 0.00
Probability of relevance for sample 577: 0.16
Probability of relevance for sample 578: 0.00
Probability of relevance for sample 579: 0.00
Probability of relevance for sample 580: 0.00
Probability of relevance for sample 581: 0.00
Probability of relevance for sample 582: 0.00
Probability of relevance for sample 583: 0.12
Probability of relevance for sample 584: 0.00
Probability of relevance for sample 585: 0.14
Probability of relevance for sample 586: 0.00
Probability of relevance for sample 587: 0.00
Probability of relevance for sample 588: 0.00
Probability of relevance for sample 589: 0.00
Probability of relevance for sample 590: 0.00
Probability of relevance for sample 591: 0.00
Probability of relevance for sample 592: 0.31
Probability of relevance for sampl

/var/folders/9j/j4gyltgs6txd83bg66_mv4fn664jzt/T/ipykernel_32169/3805266683.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_coded['relevance_probability'][i]=relevance_probability
/var/folders/9j/j4gyltgs6txd83bg66_mv4fn664jzt/T/ipyke

In [33]:
df_coded.sample(50)

,created_utc,subreddit,date_created,self,is_relevant,highly_relevant,processed_text,title,relevance_probability
291,1642636836,BPD,2022-01-20 00:00:36,1,0,0,want everything okay im always overanalyzing e...,I just want everything to be okay,4.209978e-02
427,1642828433,BPD,2022-01-22 05:13:53,1,0,0,ever life happiness stability biggest fear sta...,how can i ever have a life of happiness and st...,0.000000e+00
378,1642521632,BPD,2022-01-18 16:00:32,1,0,0,update post need perspective dont know explain...,"UPDATE ON POST ""NEED PERSPECTIVE""",0.000000e+00
467,1642780017,BPD,2022-01-21 15:46:57,1,1,0,i’ve rough couple day something someone said m...,i’ve been having a rough couple of days but so...,2.367687e-01
547,1641500935,BPD,2022-01-06 20:28:55,1,1,0,got diagnosed bpd today sure year super valida...,i got diagnosed with BPD today,4.409053e-01
125,1643654434,BPD,2022-01-31 18:40:34,1,1,0,20f shame overreacting need advice recently di...,[20F] shame for overreacting. i need advices (...,1.892253e-03
228,1642337843,BPD,2022-01-16 12:57:23,1,0,0,make fp connection healthier fp best friend go...,How can I make my FP connection healthier?,0.000000e+00
256,1642293200,BPD,2022-01-16 00:33:20,1,0,0,anger feel good dont know sometimes fantasize ...,Anger feels so good to me. I don't know why,0.000000e+00
485,1641579351,BPD,2022-01-07 18:15:51,1,0,0,partner lied bpd become codependent partner li...,Partner lied to me,0.000000e+00
594,1643580635,BPD,2022-01-30 22:10:35,1,0,0,vent nobody want talk feel isolated scared goi...,Vent.,0.000000e+00
